In [28]:
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', 10000)

In [2]:
cpi_data = pd.read_csv("../data/external/cpi/CPI_2018-2020.csv")

In [21]:
can_cpi = cpi_data[1:37]
can_cpi

,LOCATION,INDICATOR,SUBJECT,MEASURE,FREQUENCY,TIME,Value,Flag Codes
1,CAN,CPI,TOT,AGRWTH,M,2018-01,1.698842,NaN
2,CAN,CPI,TOT,AGRWTH,M,2018-02,2.158828,NaN
3,CAN,CPI,TOT,AGRWTH,M,2018-03,2.309469,NaN
4,CAN,CPI,TOT,AGRWTH,M,2018-04,2.223926,NaN
5,CAN,CPI,TOT,AGRWTH,M,2018-05,2.222222,NaN
6,CAN,CPI,TOT,AGRWTH,M,2018-06,2.453988,NaN
7,CAN,CPI,TOT,AGRWTH,M,2018-07,2.990798,NaN
8,CAN,CPI,TOT,AGRWTH,M,2018-08,2.835249,NaN
9,CAN,CPI,TOT,AGRWTH,M,2018-09,2.217125,NaN
10,CAN,CPI,TOT,AGRWTH,M,2018-10,2.444614,NaN


In [24]:
jpn_cpi = cpi_data[38:74]
jpn_cpi

,LOCATION,INDICATOR,SUBJECT,MEASURE,FREQUENCY,TIME,Value,Flag Codes
38,JPN,CPI,TOT,AGRWTH,M,2018-01,1.4,NaN
39,JPN,CPI,TOT,AGRWTH,M,2018-02,1.5,NaN
40,JPN,CPI,TOT,AGRWTH,M,2018-03,1.1,NaN
41,JPN,CPI,TOT,AGRWTH,M,2018-04,0.6,NaN
42,JPN,CPI,TOT,AGRWTH,M,2018-05,0.7,NaN
43,JPN,CPI,TOT,AGRWTH,M,2018-06,0.7,NaN
44,JPN,CPI,TOT,AGRWTH,M,2018-07,0.9,NaN
45,JPN,CPI,TOT,AGRWTH,M,2018-08,1.3,NaN
46,JPN,CPI,TOT,AGRWTH,M,2018-09,1.2,NaN
47,JPN,CPI,TOT,AGRWTH,M,2018-10,1.4,NaN


In [30]:
chf_cpi = cpi_data[75:111]
chf_cpi

,LOCATION,INDICATOR,SUBJECT,MEASURE,FREQUENCY,TIME,Value,Flag Codes
75,CHE,CPI,TOT,AGRWTH,M,2018-01,0.740203,NaN
76,CHE,CPI,TOT,AGRWTH,M,2018-02,0.628030,NaN
77,CHE,CPI,TOT,AGRWTH,M,2018-03,0.795368,NaN
78,CHE,CPI,TOT,AGRWTH,M,2018-04,0.803191,NaN
79,CHE,CPI,TOT,AGRWTH,M,2018-05,0.999232,NaN
80,CHE,CPI,TOT,AGRWTH,M,2018-06,1.146528,NaN
81,CHE,CPI,TOT,AGRWTH,M,2018-07,1.230351,NaN
82,CHE,CPI,TOT,AGRWTH,M,2018-08,1.180957,NaN
83,CHE,CPI,TOT,AGRWTH,M,2018-09,0.989465,NaN
84,CHE,CPI,TOT,AGRWTH,M,2018-10,1.106566,NaN


In [34]:
gbp_cpi = cpi_data[112:148]
gbp_cpi

,LOCATION,INDICATOR,SUBJECT,MEASURE,FREQUENCY,TIME,Value,Flag Codes
112,GBR,CPI,TOT,AGRWTH,M,2018-01,2.7,NaN
113,GBR,CPI,TOT,AGRWTH,M,2018-02,2.5,NaN
114,GBR,CPI,TOT,AGRWTH,M,2018-03,2.3,NaN
115,GBR,CPI,TOT,AGRWTH,M,2018-04,2.2,NaN
116,GBR,CPI,TOT,AGRWTH,M,2018-05,2.3,NaN
117,GBR,CPI,TOT,AGRWTH,M,2018-06,2.3,NaN
118,GBR,CPI,TOT,AGRWTH,M,2018-07,2.3,NaN
119,GBR,CPI,TOT,AGRWTH,M,2018-08,2.4,NaN
120,GBR,CPI,TOT,AGRWTH,M,2018-09,2.2,NaN
121,GBR,CPI,TOT,AGRWTH,M,2018-10,2.2,NaN


In [36]:
usd_cpi = cpi_data[149:185]
usd_cpi

,LOCATION,INDICATOR,SUBJECT,MEASURE,FREQUENCY,TIME,Value,Flag Codes
149,USA,CPI,TOT,AGRWTH,M,2018-01,2.070508,NaN
150,USA,CPI,TOT,AGRWTH,M,2018-02,2.211795,NaN
151,USA,CPI,TOT,AGRWTH,M,2018-03,2.359711,NaN
152,USA,CPI,TOT,AGRWTH,M,2018-04,2.462744,NaN
153,USA,CPI,TOT,AGRWTH,M,2018-05,2.801012,NaN
154,USA,CPI,TOT,AGRWTH,M,2018-06,2.871548,NaN
155,USA,CPI,TOT,AGRWTH,M,2018-07,2.949515,NaN
156,USA,CPI,TOT,AGRWTH,M,2018-08,2.699180,NaN
157,USA,CPI,TOT,AGRWTH,M,2018-09,2.276972,NaN
158,USA,CPI,TOT,AGRWTH,M,2018-10,2.522470,NaN


In [40]:
eur_cpi = cpi_data[223:259]
eur_cpi

,LOCATION,INDICATOR,SUBJECT,MEASURE,FREQUENCY,TIME,Value,Flag Codes
223,EA19,CPI,TOT,AGRWTH,M,2018-01,1.3,NaN
224,EA19,CPI,TOT,AGRWTH,M,2018-02,1.1,NaN
225,EA19,CPI,TOT,AGRWTH,M,2018-03,1.4,NaN
226,EA19,CPI,TOT,AGRWTH,M,2018-04,1.2,NaN
227,EA19,CPI,TOT,AGRWTH,M,2018-05,2.0,NaN
228,EA19,CPI,TOT,AGRWTH,M,2018-06,2.0,NaN
229,EA19,CPI,TOT,AGRWTH,M,2018-07,2.2,NaN
230,EA19,CPI,TOT,AGRWTH,M,2018-08,2.1,NaN
231,EA19,CPI,TOT,AGRWTH,M,2018-09,2.1,NaN
232,EA19,CPI,TOT,AGRWTH,M,2018-10,2.3,NaN


In [9]:
australia_cpi_data = pd.read_csv("../data/external/cpi/AUD_CPI_2018-2020.csv")
australia_cpi_data

,Quarter,Change from previous quarter (%)
0,Sep-11,0.6
1,Dec-11,0.0
2,Mar-12,0.1
3,Jun-12,0.5
4,Sep-12,1.4
5,Sep-12,0.2
6,Mar-13,0.4
7,Jun-13,0.4
8,Sep-13,1.2
9,Sep-13,0.8


In [42]:
aud_cpi = australia_cpi_data[25:37]
aud_cpi

,Quarter,Change from previous quarter (%)
25,Dec-17,0.6
26,Mar-18,0.4
27,Jun-18,0.4
28,Sep-18,0.4
29,Dec-18,0.5
30,Mar-19,0.0
31,Jun-19,0.6
32,Sep-19,0.5
33,Dec-19,0.7
34,Mar-20,0.3


In [11]:
new_zealand_cpi_data = pd.read_csv("../data/external/cpi/NZD_CPI_2018-2020.csv")

In [12]:
new_zealand_cpi_data

,Year ended,Percentage change
0,Dec-16,1.3
1,Mar-17,2.2
2,Jun-17,1.7
3,Sep-17,1.9
4,Dec-17,1.6
5,Mar-18,1.1
6,Jun-18,1.5
7,Sep-18,1.9
8,Dec-18,1.9
9,Mar-19,1.5


In [13]:
nzd_cpi = new_zealand_cpi_data

In [16]:
nzd_cpi = nzd_cpi.rename(columns={"Year ended": "Time", "Percentage change": "CPI"})
aud_cpi = aud_cpi.rename(columns={"Quarter": "Time", "Change from previous quarter (%)": "CPI"})
usd_cpi = usd_cpi.rename(columns={"TIME": "Time", "Value": "CPI"})
cad_cpi = cad_cpi.rename(columns={"TIME": "Time", "Value": "CPI"})
eur_cpi = eur_cpi.rename(columns={"TIME": "Time", "Value": "CPI"})
chf_cpi = chf_cpi.rename(columns={"TIME": "Time", "Value": "CPI"})
jpn_cpi = jpn_cpi.rename(columns={"TIME": "Time", "Value": "CPI"})
gbp_cpi = gbp_cpi.rename(columns={"TIME": "Time", "Value": "CPI"})

usd_cpi = usd_cpi[{"Time", "CPI"}]
cad_cpi = cad_cpi[{"Time", "CPI"}]
aud_cpi = aud_cpi[{"Time", "CPI"}]
eur_cpi = eur_cpi[{"Time", "CPI"}]
nzd_cpi = nzd_cpi[{"Time", "CPI"}]
chf_cpi = chf_cpi[{"Time", "CPI"}]
jpn_cpi = jpn_cpi[{"Time", "CPI"}]
gbp_cpi = gbp_cpi[{"Time", "CPI"}]

In [46]:
from datetime import datetime

def filled_dataframe(currency):
    time_array = pd.date_range(start="1/1/2018", freq="1H", periods=26304)
    cpi_array = np.zeros(26304)
    cpi_index = 0
    index = 0
    previous = 0
    for i in currency.iterrows():
        currency_datetime = datetime.strptime(i[1]["Time"], "%Y-%m")
        while index < 26304 and cpi_index < 26304 and time_array[index] < currency_datetime:
            cpi_array[cpi_index] = previous
            cpi_index += 1
            index += 1
        previous = i[1]["CPI"]
    while cpi_index < 26304:
        cpi_array[cpi_index] = currency["CPI"].iloc[-1]
        cpi_index += 1
    new_df = pd.DataFrame({"Time": time_array, "CPI": cpi_array})
    return new_df

def cpi_dataframe():
    cpi_data = pd.read_csv("../data/external/cpi/CPI_2018-2020.csv")
    cpi_data = cpi_data.rename(columns={"TIME": "Time", "Value": "CPI"})
    cpi_data = cpi_data[{"Time", "CPI"}]
    cpi_data["Time"] = cpi_data["Time"].transform(lambda time : datetime.strptime(time, "%Y-%m").strftime("%Y-%m-%d %H:%M"))
    
    time_frame = pd.date_range(start="2018-01-01 22:00:00", freq="1T", end="2020-12-31 21:59:00")
    time_frame = pd.DataFrame(time_frame, columns=["Time"])
    time_frame["Time"] = time_frame["Time"].dt.strftime("%Y-%m-%d %H:%M")
    
    can_cpi = cpi_data[1:37]
    jpn_cpi = cpi_data[38:74]
    chf_cpi = cpi_data[75:111]
    gbp_cpi = cpi_data[112:148]
    usd_cpi = cpi_data[149:185]
    eur_cpi = cpi_data[223:259]
    
    australia_cpi_data = pd.read_csv("../data/external/cpi/AUD_CPI_2018-2020.csv")
    new_zealand_cpi_data = pd.read_csv("../data/external/cpi/NZD_CPI_2018-2020.csv")
    aud_cpi = australia_cpi_data[25:37]
    nzd_cpi = new_zealand_cpi_data[4:]
    nzd_cpi = nzd_cpi.rename(columns={"Year ended": "Time", "Percentage change": "CPI"})
    aud_cpi = aud_cpi.rename(columns={"Quarter": "Time", "Change from previous quarter (%)": "CPI"})
    aud_cpi["Time"] = aud_cpi["Time"].transform(lambda time : datetime.strptime(time[:4] + "20" + time[4:], "%b-%Y").strftime("%Y-%m-%d %H:%M"))
    nzd_cpi["Time"] = nzd_cpi["Time"].transform(lambda time : datetime.strptime(time[:4] + "20" + time[4:], "%b-%Y").strftime("%Y-%m-%d %H:%M"))
    
    create_cpi_csv(aud_cpi, time_frame, aud_cpi["CPI"].iloc[0]).to_csv("../data/processed/cpi/aud_cpi_processed.csv")
    create_cpi_csv(can_cpi, time_frame, can_cpi["CPI"].iloc[0]).to_csv("../data/processed/cpi/can_cpi_processed.csv")
    create_cpi_csv(jpn_cpi, time_frame, jpn_cpi["CPI"].iloc[0]).to_csv("../data/processed/cpi/jpn_cpi_processed.csv")
    create_cpi_csv(chf_cpi, time_frame, chf_cpi["CPI"].iloc[0]).to_csv("../data/processed/cpi/chf_cpi_processed.csv")
    create_cpi_csv(gbp_cpi, time_frame, gbp_cpi["CPI"].iloc[0]).to_csv("../data/processed/cpi/gbp_cpi_processed.csv")
    create_cpi_csv(usd_cpi, time_frame, usd_cpi["CPI"].iloc[0]).to_csv("../data/processed/cpi/usd_cpi_processed.csv")
    create_cpi_csv(eur_cpi, time_frame, eur_cpi["CPI"].iloc[0]).to_csv("../data/processed/cpi/eur_cpi_processed.csv")
    create_cpi_csv(nzd_cpi, time_frame, nzd_cpi["CPI"].iloc[0]).to_csv("../data/processed/cpi/nzd_cpi_processed.csv")
    
def create_cpi_csv(pair, time, initial):
    pair = time.merge(pair, how="left", on="Time")
    pair["CPI"].iloc[0] = initial
    pair = pair.fillna(method="ffill")
    return pair

cpi_dataframe()

,Time,CPI
0,2018-01-01 22:00,0.6
1,2018-01-01 22:01,0.6
2,2018-01-01 22:02,0.6
3,2018-01-01 22:03,0.6
4,2018-01-01 22:04,0.6
...,...,...
1576795,2020-12-31 21:55,1.6
1576796,2020-12-31 21:56,1.6
1576797,2020-12-31 21:57,1.6
1576798,2020-12-31 21:58,1.6
